# Using `pybind11`

The package `pybind11` is provides an elegant way to wrap C++ code for Python, including automatic conversions for `numpy` arrays and the C++ `Eigen` linear algebra library. Used with the `cppimport` package, this provides a very nice work flow for integrating C++ and Python:

- Edit C++ code
- Run Python code

```bash
pip install pybind11
pip install cppimport
```

Clone the Eigen library - no installation is required as Eigen is a header only library.

```
hg clone https://bitbucket.org/eigen/eigen/
```

## Resources

- [`pybind11`](http://pybind11.readthedocs.io/en/latest/)
- [`cppimport`](https://github.com/tbenthompson/cppimport)
- [`Eigen`](http://eigen.tuxfamily.org)

## A first example of using `pybind11`

Create a new subdirectory - e.g. `example1` and create the following 5 files in it:

- `funcs.hpp`
- `funcs.cpp`
- `wrap.cpp`
- `setup.py`
- `test_funcs.py`

First write the C++ header and implementation files

In [1]:
%mkdir example1
%cd example1

/Users/cliburn/git-teach/sta-663-2017-public/scratch/example1


In [2]:
%%file funcs.hpp

int add(int i, int j);

Writing funcs.hpp


In [3]:
%%file funcs.cpp

int add(int i, int j) {
    return i + j;
};

Writing funcs.cpp


Next write the C++ wrapper code using `pybind11` in `wrap.cpp`. The arguments `"i"_a=1, "j"_a=2` in the exported function definition tells `pybind11` to generate variables named `i` with default value 1 and `j` with default value 2 for the `add` function.

In [4]:
%%file wrap.cpp
#include <pybind11/pybind11.h>
#include "funcs.hpp"

namespace py = pybind11;

using namespace pybind11::literals;

PYBIND11_PLUGIN(wrap) {
    py::module m("wrap", "pybind11 example plugin");
    m.def("add", &add, "A function which adds two numbers",
          "i"_a=1, "j"_a=2);
    return m.ptr();
}

Writing wrap.cpp


Finally, write the `setup.py` file to compile the extension module. This is mostly boilerplate.

In [5]:
%%file setup.py
import os, sys

from distutils.core import setup, Extension
from distutils import sysconfig

cpp_args = ['-std=c++11', '-stdlib=libc++', '-mmacosx-version-min=10.7']

ext_modules = [
    Extension(
    'wrap',
        ['funcs.cpp', 'wrap.cpp'],
        include_dirs=['pybind11/include'],
    language='c++',
    extra_compile_args = cpp_args,
    ),
]

setup(
    name='wrap',
    version='0.0.1',
    author='Cliburn Chan',
    author_email='cliburn.chan@duke.edu',
    description='Example',
    ext_modules=ext_modules,
)

Writing setup.py


Now build the extension module in the subdirectory with these files

In [6]:
%%bash

python setup.py build_ext -i

running build_ext
building 'wrap' extension
creating build
creating build/temp.macosx-10.6-x86_64-3.5
/usr/bin/clang -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/cliburn/anaconda2/envs/p3/include -arch x86_64 -Ipybind11/include -I/Users/cliburn/anaconda2/envs/p3/include/python3.5m -c funcs.cpp -o build/temp.macosx-10.6-x86_64-3.5/funcs.o -std=c++11 -stdlib=libc++ -mmacosx-version-min=10.7
/usr/bin/clang -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/cliburn/anaconda2/envs/p3/include -arch x86_64 -Ipybind11/include -I/Users/cliburn/anaconda2/envs/p3/include/python3.5m -c wrap.cpp -o build/temp.macosx-10.6-x86_64-3.5/wrap.o -std=c++11 -stdlib=libc++ -mmacosx-version-min=10.7
/usr/bin/clang++ -bundle -undefined dynamic_lookup -L/Users/cliburn/anaconda2/envs/p3/lib -arch x86_64 build/temp.macosx-10.6-x86_64-3.5/funcs.o build/temp.macosx-10.6-x86_64-3.5/wrap.o 

clang: warning: libstdc++ is deprecated; move to libc++ with a minimum deployment target of OS X 10.9
ld: warning: object file (build/temp.macosx-10.6-x86_64-3.5/funcs.o) was built for newer OSX version (10.7) than being linked (10.6)
ld: warning: object file (build/temp.macosx-10.6-x86_64-3.5/wrap.o) was built for newer OSX version (10.7) than being linked (10.6)


And if you are successful, you should now see a new ```funcs.so``` extension module. We can write a `test_funcs.py` file test the extension module:

In [7]:
%%file test_funcs.py

import wrap

def test_add():
    print(wrap.add(3, 4))
    assert(wrap.add(3, 4) == 7)

if __name__ == '__main__':
    test_add()

Writing test_funcs.py


And finally, running the test should not generate any error messages:

In [8]:
%%bash

python test_funcs.py

7


In [9]:
%cd ..

/Users/cliburn/git-teach/sta-663-2017-public/scratch


## Using `cppimport`

In the development stage, it can be distracting to have to repeatedly rebuild the extension module by running

```bash
python setup.py clean
python setup.py build_ext -i
```

every single time you modify the C++ code. The `cppimport` package does this for you. 

Create a new sub-directory `exaample2` and copy the files `func.hpp`, `funcs.cpp` and `wrap.cpp` from `example1` over.
For the previous example, we just need to add some annotation (between `<% and %>` delimiters) to the top of the `wrap.cpp` file

In [10]:
%mkdir example2
%cp example1/funcs.* example2/
%cd example2

/Users/cliburn/git-teach/sta-663-2017-public/scratch/example2


In [11]:
%%file wrap.cpp
<%
cfg['compiler_args'] = ['-std=c++11', '-stdlib=libc++', '-mmacosx-version-min=10.7']
cfg['sources'] = ['funcs.cpp']
setup_pybind11(cfg)
%>

#include "funcs.hpp"
#include <pybind11/pybind11.h>

namespace py = pybind11;

PYBIND11_PLUGIN(wrap) {
    py::module m("wrap", "pybind11 example plugin");
    m.def("add", &add, "A function which adds two numbers");
    return m.ptr();
}

Writing wrap.cpp


In [12]:
%%file test_funcs.py

import cppimport
funcs = cppimport.imp("wrap")

def test_add():
    assert(funcs.add(3, 4) == 7)

if __name__ == '__main__':
    print(funcs.add(3,4))
    test_add()

Writing test_funcs.py


In [13]:
%%bash

python test_funcs.py 

7


clang: warning: libstdc++ is deprecated; move to libc++ with a minimum deployment target of OS X 10.9
ld: warning: object file (/var/folders/xf/rzdg30ps11g93j3w0h589q780000gn/T/tmp5fz61n6k/Users/cliburn/git-teach/sta-663-2017-public/scratch/example2/funcs.o) was built for newer OSX version (10.7) than being linked (10.6)
ld: warning: object file (/var/folders/xf/rzdg30ps11g93j3w0h589q780000gn/T/tmp5fz61n6k/Users/cliburn/git-teach/sta-663-2017-public/scratch/example2/.rendered.wrap.o) was built for newer OSX version (10.7) than being linked (10.6)


Or just call from notebook

In [14]:
import cppimport
funcs = cppimport.imp("wrap")

funcs.add(3, 4)

{'In': ['',
  "get_ipython().magic('mkdir example1')\nget_ipython().magic('cd example1')",
  "get_ipython().run_cell_magic('file', 'funcs.hpp', '\\nint add(int i, int j);')",
  "get_ipython().run_cell_magic('file', 'funcs.cpp', '\\nint add(int i, int j) {\\n    return i + j;\\n};')",
  'get_ipython().run_cell_magic(\'file\', \'wrap.cpp\', \'#include <pybind11/pybind11.h>\\n#include "funcs.hpp"\\n\\nnamespace py = pybind11;\\n\\nusing namespace pybind11::literals;\\n\\nPYBIND11_PLUGIN(wrap) {\\n    py::module m("wrap", "pybind11 example plugin");\\n    m.def("add", &add, "A function which adds two numbers",\\n          "i"_a=1, "j"_a=2);\\n    return m.ptr();\\n}\')',
  'get_ipython().run_cell_magic(\'file\', \'setup.py\', "import os, sys\\n\\nfrom distutils.core import setup, Extension\\nfrom distutils import sysconfig\\n\\ncpp_args = [\'-std=c++11\', \'-stdlib=libc++\', \'-mmacosx-version-min=10.7\']\\n\\next_modules = [\\n    Extension(\\n    \'wrap\',\\n        [\'funcs.cpp\', \'wra

without any need to manually build the extension module. Any updates will be detected by `cppimport` and it will automatically trigger a re-build.

In [15]:
%cd ..

/Users/cliburn/git-teach/sta-663-2017-public/scratch


## Vectorizing functions for use with `numpy` arrays

Example showing how to vectorize a `square` function. Note that from here on, we don't bother to use separate header and implementation files for these code snippets, and just write them together with the wrapping code in a `code.cpp` file. This means that with `cppimport`, there are only two files that we actually code for, a C++ `code.cpp` file and a python test file.

In [16]:
%mkdir example3
%cd example3

/Users/cliburn/git-teach/sta-663-2017-public/scratch/example3


In [17]:
%%file wrap.cpp
<%
cfg['compiler_args'] = ['-std=c++11', '-stdlib=libc++', '-mmacosx-version-min=10.7']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>

namespace py = pybind11;

double square(double x) {
    return x * x;
}

PYBIND11_PLUGIN(code) {
    py::module m("wrap", "auto-compiled c++ extension");
    m.def("square", py::vectorize(square));
    return m.ptr();
}

Writing wrap.cpp


In [18]:
import cppimport
import numpy as np

wrap = cppimport.imp("wrap")

xs = np.random.rand(10)
print(xs)
print(wrap.square(xs))

ys = range(10)
print(wrap.square(ys))

[ 0.11452981  0.54149398  0.54458795  0.04748548  0.26545907  0.52472784
  0.55701113  0.38717192  0.76610652  0.89159453]


AttributeError: module 'wrap' has no attribute 'square'

In [ ]:
%cd ..

## Using `numpy` arrays as function arguments and return values

Example showing how to pass `numpy` arrays in and out of functions. These `numpy` array arguments can either be generic `py:array` or typed `py:array_t<double>`. The properties of the `numpy` array can be obtained by calling its `request` method. This returns a `struct` of the following form:

```c++
struct buffer_info {
    void *ptr;
    size_t itemsize;
    std::string format;
    int ndim;
    std::vector<size_t> shape;
    std::vector<size_t> strides;
};
```

Here is C++ code for two functions - the function `twice` shows how to change a passed in `numpy` array in-place using pointers; the function `sum` shows how to sum the elements of a `numpy` array. By taking advantage of the information in `buffer_info`, the code will work for arbitrary `n-d` arrays.

In [ ]:
%mkdir example4
%cd example4

In [ ]:
%%file code.cpp
<%
cfg['compiler_args'] = ['-std=c++11', '-stdlib=libc++', '-mmacosx-version-min=10.7']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>

namespace py = pybind11;

// Passing in an array of doubles
void twice(py::array_t<double> xs) {
    py::buffer_info info = xs.request();
    auto ptr = static_cast<double *>(info.ptr);

    int n = 1;
    for (auto r: info.shape) {
      n *= r;
    }

    for (int i = 0; i <n; i++) {
        *ptr++ *= 2;
    }
}

// Passing in a generic array
double sum(py::array xs) {
    py::buffer_info info = xs.request();
    auto ptr = static_cast<double *>(info.ptr);

    int n = 1;
    for (auto r: info.shape) {
      n *= r;
    }

    double s = 0.0;
    for (int i = 0; i <n; i++) {
        s += *ptr++;
    }

    return s;
}

PYBIND11_PLUGIN(code) {
    pybind11::module m("code", "auto-compiled c++ extension");
    m.def("sum", &sum);
    m.def("twice", &twice);
    return m.ptr();
}

In [ ]:
%%file test_code.py
import cppimport
import numpy as np

code = cppimport.imp("code")

if __name__ == '__main__':
    xs = np.arange(12).reshape(3,4).astype('float')
    print(xs)
    print("np :", xs.sum())
    print("cpp:", code.sum(xs))

    print()
    code.twice(xs)
    print(xs)

In [ ]:
%%bash

python test_code.py

In [ ]:
%cd ..

## More on working with `numpy` arrays

This example shows how to use array access for `numpy` arrays within the C++ function. It is taken from the `pybind11` documentation, but fixes a small bug in the official version. As noted in the documentation, the function would be more easily coded using `py::vectorize`.

```c++
<%
cfg['compiler_args'] = ['-std=c++11', '-stdlib=libc++', '-mmacosx-version-min=10.7']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>

namespace py = pybind11;

py::array_t<double> add_arrays(py::array_t<double> input1, py::array_t<double> input2) {
    auto buf1 = input1.request(), buf2 = input2.request();

    if (buf1.ndim != 1 || buf2.ndim != 1)
        throw std::runtime_error("Number of dimensions must be one");

    if (buf1.shape[0] != buf2.shape[0])
        throw std::runtime_error("Input shapes must match");

    auto result = py::array(py::buffer_info(
        nullptr,            /* Pointer to data (nullptr -> ask NumPy to allocate!) */
        sizeof(double),     /* Size of one item */
        py::format_descriptor<double>::value, /* Buffer format */
        buf1.ndim,          /* How many dimensions? */
        { buf1.shape[0] },  /* Number of elements for each dimension */
        { sizeof(double) }  /* Strides for each dimension */
    ));

    auto buf3 = result.request();

    double *ptr1 = (double *) buf1.ptr,
           *ptr2 = (double *) buf2.ptr,
           *ptr3 = (double *) buf3.ptr;

    for (size_t idx = 0; idx < buf1.shape[0]; idx++)
        ptr3[idx] = ptr1[idx] + ptr2[idx];

    return result;
}

PYBIND11_PLUGIN(code) {
    py::module m("code");
    m.def("add_arrays", &add_arrays, "Add two NumPy arrays");
    return m.ptr();
}
```

with test code
```python
import cppimport
import numpy as np

code = cppimport.imp("code")

if __name__ == '__main__':
    xs = np.arange(12)
    print(xs)

    print(code.add_arrays(xs, xs))
```

## Using the C++ `eigen` library to calculate matrix inverse and determinant

Example showing how `Eigen` vectors and matrices can be passed in and out of C++ functions. Note that `Eigen` arrays are automatically converted to/from `numpy` arrays simply by including the `pybind/eigen.h` header. Because of this, it is probably simplest in most cases to work with `Eigen` vectors and matrices rather than `py::buffer` or `py::array` where `py::vectorize` is insufficient.

```c++
<%
cfg['compiler_args'] = ['-std=c++11', '-stdlib=libc++', '-mmacosx-version-min=10.7']
cfg['include_dirs'] = ['/Users/cliburn/hg/eigen']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/eigen.h>

#include <Eigen/LU>

namespace py = pybind11;

// convenient matrix indexing comes for free
double get(Eigen::MatrixXd xs, int i, int j) {
    return xs(i, j);
}

// takes numpy array as input and returns double
double det(Eigen::MatrixXd xs) {
    return xs.determinant();
}

// takes numpy array as input and returns another numpy array
Eigen::MatrixXd inv(Eigen::MatrixXd xs) {
    return xs.inverse();
}

PYBIND11_PLUGIN(code) {
    pybind11::module m("code", "auto-compiled c++ extension");
    m.def("inv", &inv);
    m.def("det", &det);
    return m.ptr();
}
```

and test code

```python
import cppimport
import numpy as np

code = cppimport.imp("code")

if __name__ == '__main__':
    A = np.array([[1,2,1],
                  [2,1,0],
                  [-1,1,2]])

    print(A)
    print(code.det(A))
    print(code.inv(A))
```

## Using `pybind11` with `openmp`

Here is a standard example of using OpenMP to integrate the value of $\pi$ written using `pybind11`.

```c++
<%
cfg['compiler_args'] = ['-std=c++11', '-fopenmp']
cfg['linker_args'] = ['-fopenmp']
setup_pybind11(cfg)
%>

#include <omp.h>
#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>

namespace py = pybind11;

double calc_pi(int n) {
  /* Acquire GIL before calling Python code */
  py::gil_scoped_acquire acquire;

  int i;
  double step = 1.0/n;
  double s = 0;
  
  #pragma omp parallel
  {
    double x;
    #pragma omp for reduction(+:s) 
    for (i=0; i<n; i++) {
      x = (i+0.5) * step;
      s += 4.0/(1 + x*x);
    }
  }
  return step * s;
};

PYBIND11_PLUGIN(code) {
  pybind11::module m("code", "auto-compiled c++ extension");
  m.def("calc_pi", [](int n) {
      /* Release GIL before calling into C++ code */      
      py::gil_scoped_release release;
      return calc_pi(n);
    });

  return m.ptr();
}
```

And here is the test code.

```python
import cppimport
import numpy as np

code = cppimport.imp("code")

if __name__ == '__main__':
    n = int(1e9)

    print(code.calc_pi(n))
```
